# 导入库

In [1]:
import pandas as pd
import numpy as np
#import numpy as np#for key in sandbeachlist
from snownlp import SnowNLP
from snownlp import sentiment
from zhon.hanzi import punctuation
import string
import re
import jieba
import copy
from gensim.corpora import Dictionary
from pprint import pprint
from gensim.models.ldamodel import LdaModel
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
import warnings
warnings.filterwarnings("ignore")

D:\Anaconda3-2021.05\lib\site-packages\gensim\similarities\__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


# 定义景区名称、加载停用词

In [2]:
name = ['第一海水浴场', '青岛水族馆', '青岛海底世界', '青岛海昌极地海洋公园', '青岛海泉湾海洋之星摩天轮',
        '黄岛金沙滩', '石老人海水浴场', '第二海水浴场', '第三海水浴场', '琴岛之眼摩天轮', '栈桥', '青岛奥帆中心', '小青岛', '琅琊台风景区']
sandbeach = ['第一海水浴场', '黄岛金沙滩', '石老人海水浴场', '第二海水浴场', '第三海水浴场']
aquarium = ['青岛水族馆', '青岛海底世界', '青岛海昌极地海洋公园']
skywheel = ['青岛海泉湾海洋之星摩天轮', '琴岛之眼摩天轮']
park = ['栈桥', '青岛奥帆中心', '小青岛', '琅琊台风景区']

In [3]:
with open('stopwords.txt', 'r', newline='', encoding='utf-8') as txtfile:
    stopwords = txtfile.read().split('\r\n')

# 贝叶斯分类模型训练

In [4]:
def load_corpus(path):
    data = []
    with open(path, "r", encoding="utf8") as f:
        for line in f:
            [_, seniment, content] = line.split(",", 2)
            content = processing(content)
            data.append((content, int(seniment)))
    return data


def processing(text):
    text = re.sub("\{%.+?%\}", " ", text)
    text = re.sub("@.+?( |$)", " ", text)
    text = re.sub("【.+?】", " ", text)
    text = re.sub("\u200b", " ", text)
    words = [w for w in jieba.lcut(text) if w.isalpha()]
    while "不" in words:
        index = words.index("不")
        if index == len(words) - 1:
            break
        words[index: index+2] = ["".join(words[index: index+2])]
    result = " ".join(words)
    return result

In [5]:
TRAIN_PATH = "train.txt"
train_data = load_corpus(TRAIN_PATH)
df_train = pd.DataFrame(train_data, columns=["words", "label"])
df_train.head()
vectorizer = CountVectorizer(token_pattern='\[?\w+\]?', stop_words=stopwords)
X_train = vectorizer.fit_transform(df_train["words"])
y_train = df_train["label"]
clf = MultinomialNB()
clf.fit(X_train, y_train)

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\86178\AppData\Local\Temp\jieba.cache
Loading model cost 0.733 seconds.
Prefix dict has been built successfully.


MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

# 评论读取以及标签

In [103]:
comments = []
star_level=[]
score = []
scenic_spot=[]
for index, place in enumerate(aquarium):
    path = 'clean_data/'+place+'new_.csv'
    data = pd.read_csv(path)
    for i in range(len(data['评论'])):
        comments.append(data['评论'][i])
        star_level.append(data['得分'][i])
        scenic_spot.append(place)
        score.append(SnowNLP(data['评论'][i]).sentiments)

In [104]:
def process(comments):
    pattern = re.compile(u'\t|\n|\.|-|:|;|\)|\(|\?|"')
    punctuation_list = punctuation+string.punctuation+" "
    for pun in punctuation_list:
        comments = comments.replace(pun, '')
    comments = re.sub(pattern, ' ', comments)
    seg_list_exact = jieba.lcut(comments, cut_all=False)
    return ' '.join(seg_list_exact)

In [105]:
words = [process(s) for s in comments]
vec = vectorizer.transform(words)
grade = clf.predict(vec)

## 提取回归所用的数据

In [107]:
df = pd.DataFrame({'comments': comments, 'sentiments': score, 'grade': grade,'score':star_level,'ScenicSpot':scenic_spot})
df

,comments,sentiments,grade,score,ScenicSpot
0,非常满意,0.705347,1,5.0,青岛水族馆
1,很不错，挺好的。,0.980408,1,4.0,青岛水族馆
2,好,0.655863,1,5.0,青岛水族馆
3,挺好的,0.828425,1,5.0,青岛水族馆
4,非常好，,0.655863,1,5.0,青岛水族馆
...,...,...,...,...,...
130818,还不错 值得去一趟,0.968345,1,5.0,青岛海昌极地海洋公园
130819,速度太慢服务太差,0.044576,0,1.0,青岛海昌极地海洋公园
130820,感覺一般,0.291799,1,3.0,青岛海昌极地海洋公园
130821,人太多了啊,0.385037,1,4.0,青岛海昌极地海洋公园


In [120]:
df.to_csv('aquarium.csv',encoding='utf-8-sig',index=False)

In [108]:
sandbeach = ['第一海水浴场', '黄岛金沙滩', '石老人海水浴场', '第二海水浴场']#, '第三海水浴场'
aquarium = ['青岛水族馆', '青岛海底世界', '青岛海昌极地海洋公园']
skywheel = ['青岛海泉湾海洋之星摩天轮', '琴岛之眼摩天轮']
park = ['栈桥', '青岛奥帆中心', '小青岛', '琅琊台风景区']

sandbeachlist=['环境','交通','价格','设施','景色']
aquariumlist=['环境','交通','价格','设施','表演','服务态度','时间']#
skywheellist=['环境','交通','价格','设施','服务']
parklist=['环境','交通','价格','设施','天气']

In [109]:
def get_fea_dict(spots,spotlist,df):
    name_fea={}
    for spot in spots:
        tmp_df=df[df['ScenicSpot']==spot]
        tmp_df=tmp_df.reset_index(drop=True)
        standard=[np.mean([df['sentiments'][i] for i in range(df.shape[0]) if key in df['comments'][i]]) for key in spotlist]
        tmp_score=[np.mean([tmp_df['sentiments'][i] for i in range(tmp_df.shape[0]) if key in tmp_df['comments'][i]]) for key in spotlist]
        print(standard,tmp_score)
        ls=[]
        for i,j in zip(standard,tmp_score):
            if i<j:
                ls.append(1)
            else:
                ls.append(0)
        name_fea[spot]=ls
    return name_fea,standard,tmp_score

name_fea,standard,tmp_score=get_fea_dict(aquarium,aquariumlist,df)

In [115]:
def convert_grades(score):
    if score >= 0.7:
        return "pretty good"
    elif score <= 0.3:
        return "pretty bad"
    else:
        return "just so so"

In [116]:
df['labels'] = df['sentiments'].apply(convert_grades)
df

,comments,sentiments,grade,score,ScenicSpot,labels
0,非常满意,0.705347,1,5.0,青岛水族馆,pretty good
1,很不错，挺好的。,0.980408,1,4.0,青岛水族馆,pretty good
2,好,0.655863,1,5.0,青岛水族馆,just so so
3,挺好的,0.828425,1,5.0,青岛水族馆,pretty good
4,非常好，,0.655863,1,5.0,青岛水族馆,just so so
...,...,...,...,...,...,...
130818,还不错 值得去一趟,0.968345,1,5.0,青岛海昌极地海洋公园,pretty good
130819,速度太慢服务太差,0.044576,0,1.0,青岛海昌极地海洋公园,pretty bad
130820,感覺一般,0.291799,1,3.0,青岛海昌极地海洋公园,pretty bad
130821,人太多了啊,0.385037,1,4.0,青岛海昌极地海洋公园,just so so


In [117]:
def add_data(place_list,df):
    data_tmp=df.loc[:,['comments','score','labels','ScenicSpot']]
    data_tmp=data_tmp.reindex(columns=['comments','score','labels','ScenicSpot']+place_list, fill_value=0)
    for i in range(data_tmp.shape[0]):
        for fea_ind in range(len(place_list)):
            data_tmp[place_list[fea_ind]][i]+=name_fea[data_tmp['ScenicSpot'][i]][fea_ind]
    return data_tmp[['labels']+place_list+['score']]

In [118]:
final_data=add_data(aquariumlist,df)
final_data.to_csv("C:/Users/86178/Desktop/青岛统计建模/回归分析数据/AquariumData.csv",encoding='utf-8-sig',index=False)
final_data.to_csv('AquariumData.csv',encoding='utf-8-sig',index=False)
final_data

,labels,环境,交通,价格,设施,表演,服务态度,时间,score
0,pretty good,1,1,1,1,1,0,1,5.0
1,pretty good,1,1,1,1,1,0,1,4.0
2,just so so,1,1,1,1,1,0,1,5.0
3,pretty good,1,1,1,1,1,0,1,5.0
4,just so so,1,1,1,1,1,0,1,5.0
...,...,...,...,...,...,...,...,...,...
130818,pretty good,0,1,0,0,0,0,0,5.0
130819,pretty bad,0,1,0,0,0,0,0,1.0
130820,pretty bad,0,1,0,0,0,0,0,3.0
130821,just so so,0,1,0,0,0,0,0,4.0


# 评论分类

In [12]:
df_postive = copy.deepcopy(df[df['labels'] == 1])
df_negative = copy.deepcopy(df[df['labels'] == 0])

In [13]:
df_postive = copy.deepcopy(df[df['grade'] == 1])
df_negative = copy.deepcopy(df[df['grade'] == 0])

In [14]:
df_postive.head()

,comments,sentiments,grade,labels
0,不错接着期待还带孩子来,0.982890,1,1
1,好,0.655863,1,1
5,99号小伙子，手法很好，超酸爽。过瘾，支持！,0.997289,1,1
6,强烈推荐。多运动 多睡觉 少吃零食 后脑勺不要冒泡 [笑哈哈]嗯😊我最喜欢三月的风，四月的雨...,0.996967,1,1
7,其实已经过了很久了美团还是让评价，很不错哟，跟闺蜜玩得很开心，是个好地方，就是要做好防晒哟，推荐～,0.999643,1,1


In [15]:
comments_postive = df_postive['comments'].tolist()
comments_negative = df_negative['comments'].tolist()

In [16]:
# 输入 comments：['xcbak','hxail sfv ssf','xanil dqa']
def commens_processing(comments):
    pattern = re.compile(u'\t|\n|\.|-|:|;|\)|\(|\?|"')  # 定义正则表达式匹配模式
    punctuation_list = punctuation+string.punctuation+" "
    com = []
    for index, value in enumerate(comments):
        for pun in punctuation_list:  # 去掉标点
            value = value.replace(pun, '')
        value = re.sub(pattern, '', value)  # 将符合模式的字符去除
        seg_list_exact = jieba.lcut(value, cut_all=False)  # 精确模式分词
        comments_seg = []
        for word in seg_list_exact:
            if word not in stopwords:
                comments_seg.append(word)
        if comments_seg:
            com.append(comments_seg)
    return com

# LDA主题模型

In [17]:
def lda(text, topic=10):
    dictionary = Dictionary(text)
    doc_term_matrix = [dictionary.doc2bow(doc) for doc in text]
    ldamodel = LdaModel(doc_term_matrix, num_topics=5,
                        id2word=dictionary, passes=10)
    pprint(ldamodel.print_topics(num_topics=5, num_words=10))

In [18]:
text = commens_processing(comments_postive)
lda(text)

[(0,
  '0.015*"石老人" + 0.008*"海水浴场" + 0.008*"大海" + 0.008*"海边" + 0.008*"海水" + '
  '0.007*"夏天" + 0.007*"时间" + 0.006*"沙滩" + 0.006*"不错" + 0.006*"免费"'),
 (1,
  '0.014*"沙子" + 0.010*"沙滩" + 0.010*"孩子" + 0.008*"开心" + 0.007*"螃蟹" + 0.007*"不错" '
  '+ 0.006*"海边" + 0.006*"抓" + 0.005*"二浴" + 0.004*"海滩"'),
 (2,
  '0.015*"海水浴场" + 0.014*"八大关" + 0.011*"拍照" + 0.008*"路" + 0.008*"地方" + '
  '0.007*"海边" + 0.006*"石楼" + 0.006*"浴场" + 0.006*"推荐" + 0.005*"时间"'),
 (3,
  '0.032*"金沙滩" + 0.021*"沙滩" + 0.013*"沙子" + 0.010*"舒服" + 0.009*"黄岛" + 0.008*"吹" '
  '+ 0.008*"海风" + 0.008*"海边" + 0.008*"特别" + 0.006*"细腻"'),
 (4,
  '0.043*"海水浴场" + 0.031*"沙滩" + 0.022*"浴场" + 0.022*"海水" + 0.020*"不错" + '
  '0.019*"干净" + 0.018*"八大关" + 0.015*"沙子" + 0.012*"第一" + 0.010*"海边"')]


In [19]:
text = commens_processing(comments_negative)
lda(text)

[(0,
  '0.013*"金沙滩" + 0.012*"浴场" + 0.008*"游泳" + 0.008*"说" + 0.007*"孩子" + 0.006*"管理" '
  '+ 0.006*"地方" + 0.006*"沙滩" + 0.005*"垃圾" + 0.004*"租"'),
 (1,
  '0.010*"海水浴场" + 0.009*"不错" + 0.008*"不好" + 0.008*"地方" + 0.008*"海水" + '
  '0.007*"石老人" + 0.007*"元" + 0.006*"一点" + 0.006*"态度" + 0.006*"孩子"'),
 (2,
  '0.008*"沙滩" + 0.007*"海水" + 0.007*"孩子" + 0.007*"地方" + 0.007*"说" + 0.007*"元" + '
  '0.007*"坐" + 0.007*"游泳" + 0.006*"钱" + 0.006*"金沙滩"'),
 (3,
  '0.009*"说" + 0.007*"点" + 0.006*"特别" + 0.006*"沙滩" + 0.005*"收费" + 0.005*"沙子" + '
  '0.005*"孩子" + 0.004*"元" + 0.004*"厕所" + 0.004*"海藻"'),
 (4,
  '0.014*"浴场" + 0.010*"沙滩" + 0.009*"感觉" + 0.009*"说" + 0.009*"收费" + '
  '0.008*"海水浴场" + 0.007*"地方" + 0.006*"贵" + 0.006*"海边" + 0.005*"金沙滩"')]
